# Birds Data Processing

In [1]:
# required imports
import pandas as pd
import numpy as np
import geopandas as gpd
import logging
import os
import glob
import math
import random
import pyarrow as pa
import pyarrow.parquet as pq
import gc
from pathlib import Path
from shapely.geometry import Point, LineString, Polygon
from tqdm import tqdm
from datetime import datetime
from typing import Optional, List, Dict, Any

/Users/sooneui/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Initial Filtering & Cleaning of Ebird Data<br>


### Initial feature drop <br>
- Remove unused columns & prepare state data for transformations.

In [2]:
VA_birds = pd.read_csv("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ebd_US-VA_202001_202312_smp_relMay-2025/ebd_US-VA_202001_202312_smp_relMay-2025.txt", sep = "\t")
ME_birds = pd.read_csv("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ebd_US-ME_202001_202312_smp_relMay-2025/ebd_US-ME_202001_202312_smp_relMay-2025.txt", sep = "\t")
print("VA_birds:", VA_birds.shape), print("ME_birds:", ME_birds.shape)

/var/folders/xt/_chwp0056v52sycwjk0jctdm0000gn/T/ipykernel_34845/2349757709.py:1: DtypeWarning: Columns (9,21,23,24,33,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  VA_birds = pd.read_csv("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ebd_US-VA_202001_202312_smp_relMay-2025/ebd_US-VA_202001_202312_smp_relMay-2025.txt", sep = "\t")
/var/folders/xt/_chwp0056v52sycwjk0jctdm0000gn/T/ipykernel_34845/2349757709.py:2: DtypeWarning: Columns (9,21,23,33,39) have mixed types. Specify dtype option on import or set low_memory=False.
  ME_birds = pd.read_csv("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ebd_US-ME_202001_202312_smp_relMay-2025/ebd_US-ME_202001_202312_smp_relMay-2025.txt", sep = "\t")


VA_birds: (14875315, 53)
ME_birds: (6697506, 53)


(None, None)

In [3]:
VA_birds.columns

Index(['GLOBAL UNIQUE IDENTIFIER', 'LAST EDITED DATE', 'TAXONOMIC ORDER',
       'CATEGORY', 'TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
       'SUBSPECIES COMMON NAME', 'SUBSPECIES SCIENTIFIC NAME', 'EXOTIC CODE',
       'OBSERVATION COUNT', 'BREEDING CODE', 'BREEDING CATEGORY',
       'BEHAVIOR CODE', 'AGE/SEX', 'COUNTRY', 'COUNTRY CODE', 'STATE',
       'STATE CODE', 'COUNTY', 'COUNTY CODE', 'IBA CODE', 'BCR CODE',
       'USFWS CODE', 'ATLAS BLOCK', 'LOCALITY', 'LOCALITY ID', 'LOCALITY TYPE',
       'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
       'TIME OBSERVATIONS STARTED', 'OBSERVER ID', 'OBSERVER ORCID ID',
       'SAMPLING EVENT IDENTIFIER', 'OBSERVATION TYPE', 'PROTOCOL NAME',
       'PROTOCOL CODE', 'PROJECT NAMES', 'PROJECT IDENTIFIERS',
       'DURATION MINUTES', 'EFFORT DISTANCE KM', 'EFFORT AREA HA',
       'NUMBER OBSERVERS', 'ALL SPECIES REPORTED', 'GROUP IDENTIFIER',
       'HAS MEDIA', 'APPROVED', 'REVIEWED', 'REASON', 'CHECKLIST COMMENTS',
       'SPECI

In [4]:
##Selecting relevant columns
VA_birds=VA_birds[['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME', 'OBSERVATION COUNT',
       'STATE CODE','COUNTY', 'LOCALITY', 'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 
       'OBSERVATION DATE', 'TIME OBSERVATIONS STARTED', 'DURATION MINUTES']].copy()
ME_birds=ME_birds[['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME', 'OBSERVATION COUNT',
       'STATE CODE','COUNTY', 'LOCALITY', 'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 
       'OBSERVATION DATE', 'TIME OBSERVATIONS STARTED', 'DURATION MINUTES']].copy()
print("VA:", VA_birds.shape, "ME:", ME_birds.shape)

VA: (14875315, 13) ME: (6697506, 13)


### Filter each state by study area polygons<br>
Requires 'ME_polygon.json' and 'VA_polygon.json': designated study area.

In [5]:
# Functions to Filter observations within pre-defined polygon
def filter_df_within_polygon(df, polygon_geojson, lat_col='LATITUDE', lon_col='LONGITUDE'):
    try:
        import geopandas as gpd
        from shapely.geometry import Point, Polygon
        
        # Extract polygon coordinates
        polygon_coords = polygon_geojson['features'][0]['geometry']['coordinates'][0]
        polygon = Polygon(polygon_coords)
        
        # Create GeoDataFrame - use 'df' parameter, not 'df_clean'
        geometry = [Point(lon, lat) for lon, lat in zip(df[lon_col], df[lat_col])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry)
        
        # Filter using spatial operation - removed trailing period
        filtered_gdf = gdf[gdf.geometry.within(polygon)]
        
        # Return as regular DataFrame (drop geometry column)
        return filtered_gdf.drop(columns=['geometry'])
        
    except ImportError:
        print("GeoPandas not installed. Using fallback method.")




In [6]:
VA_polygon = pd.read_json('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/VA_polygon.json')
ME_polygon = pd.read_json('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/ME_polygon.json')

In [7]:
# Filter VA by Polygon
VA_birds_filtered = filter_df_within_polygon(VA_birds, VA_polygon)
VA_birds_filtered.shape


(43098, 13)

In [8]:
# Filter ME by Polygon
ME_birds_filtered = filter_df_within_polygon(ME_birds, ME_polygon)
ME_birds_filtered.shape

(908976, 13)

### Combine data into a single df/file

In [9]:
birds=pd.concat([VA_birds_filtered, ME_birds_filtered], axis=0, ignore_index=True)
len(birds)

952074

In [10]:
birds.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/stations.parquet', compression='snappy') 

## Add Derived Features and Clean Missing Data

### Add station_distance & assigned_station
Adds following features to the df:
- station_distance: str listing stations within the defined polygon and the distances to each station from the bird observation coordiante
- assigned_station: str containing the nearest noise detection station from the bird observation coordinate<br>

Requires following files
- "I95_stations_master": A file listing all noise detection stations along/near I95 highway is required 
- "stations_in_polygon": A file listing noise detection stations within the study area

In [11]:
stations_in_polygons = pd.read_parquet("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/stations_in_polygons.parquet") 
print(stations_in_polygons.shape)
print(stations_in_polygons.columns)

(70, 6)
Index(['station_id', 'longitude', 'latitude', 'state', 'geometry',
       'index_right'],
      dtype='object')


In [12]:
# Define stations_list
stations_list = stations_in_polygons[['station_id', 'latitude', 'longitude']].to_dict('records')

In [13]:
# Function to calculate distance between two points using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Calculate the great-circle distance between two points on the Earth
    R = 6371.0  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(float, (lat1, lon1, lat2, lon2))
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def add_station_distances_and_assignment(pandas_df, stations, lat_col='LATITUDE', lon_col='LONGITUDE', id_col='station_id'):

    # stations should be a list of dicts with keys: id_col, latitude, longitude
    station_ids = [str(st[id_col]) for st in stations]
    station_coords = [(float(st['latitude']), float(st['longitude'])) for st in stations]

    # Calculate distances for each row
    def calculate_distances(row):
        try:
            if pd.isna(row[lat_col]) or pd.isna(row[lon_col]):
                print(f"Found NA values: lat={row[lat_col]}, lon={row[lon_col]}")
                return None

            distances = {}
            for station_id, (st_lat, st_lon) in zip(station_ids, station_coords):
                try:
                    # Convert to float explicitly and handle any string formatting
                    lat1 = float(str(row[lat_col]).strip())
                    lon1 = float(str(row[lon_col]).strip())
                    dist = haversine(lat1, lon1, st_lat, st_lon)
                    distances[station_id] = dist
                except Exception as e:
                    print(f"Error calculating distance for station {station_id}: {str(e)}")
                    print(f"Values: lat1={row[lat_col]}, lon1={row[lon_col]}, st_lat={st_lat}, st_lon={st_lon}")
                    distances[station_id] = None
            return distances
        except Exception as e:
            print(f"Error in calculate_distances: {str(e)}")
            return None

    # Add distances dictionary column
    pandas_df = pandas_df.copy()
    pandas_df['station_distances'] = pandas_df.apply(calculate_distances, axis=1)

    # Find closest station
    def assign_station(distances_dict):
        if not distances_dict:
            return None

        min_dist = float('inf')
        min_stations = []

        for station_id, dist in distances_dict.items():
            if dist is None:
                continue
            if dist < min_dist:
                min_dist = dist
                min_stations = [station_id]
            elif dist == min_dist:
                min_stations.append(station_id)

        if not min_stations:
            return None

        # If tie, pick randomly
        return random.choice(min_stations)

    # Add assigned station column
    pandas_df['assigned_station'] = pandas_df['station_distances'].apply(assign_station)

    return pandas_df


In [14]:
print("df shape before adding station distances:", birds.shape)
birds = add_station_distances_and_assignment(birds, stations_list, lat_col='LATITUDE', lon_col='LONGITUDE', id_col='station_id')
print("df shape after adding station distances:", birds.shape)

df shape before adding station distances: (952074, 13)
df shape after adding station distances: (952074, 15)


In [15]:
birds.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/stations.parquet', compression='snappy')  

In [16]:
na_count = birds.isna().sum()
print(f"Number of rows with NA in 'station_distances': {na_count}")

Number of rows with NA in 'station_distances': TAXON CONCEPT ID                 0
COMMON NAME                      0
SCIENTIFIC NAME                  0
OBSERVATION COUNT                0
STATE CODE                       0
COUNTY                           0
LOCALITY                         0
LOCALITY TYPE                    0
LATITUDE                         0
LONGITUDE                        0
OBSERVATION DATE                 0
TIME OBSERVATIONS STARTED     3321
DURATION MINUTES             62586
station_distances                0
assigned_station                 0
dtype: int64


### Add i95_distances <br>
Adds i95_distance feature to the df.<br>
Requires "i95_coordinates.csv", a file containing i95 coordinates with road sequence.

In [17]:
# read in the i95 coordinates
# This file contains the coordinates of the I-95 highway for the analysis
i95_coordinates = pd.read_csv('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/i95_coordinates.csv')
print(i95_coordinates.shape)
print(i95_coordinates.columns)

(55654, 10)
Index(['Way_ID', 'Segment_Number', 'Point_Order', 'Longitude', 'Latitude',
       'Highway_Type', 'Route_Ref', 'Max_Speed', 'Combined_Path',
       'Overall_Sequence'],
      dtype='object')


In [18]:
i95_sorted = i95_coordinates.sort_values(['Overall_Sequence'])
i95_coords = list(zip(i95_sorted['Latitude'], i95_sorted['Longitude']))

In [19]:
### I95 DistanceCalculator ###
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class I95DistanceCalculator:
    """
    Simplified processor to add I-95 distance column to parquet files.
    Only adds the distance column without any filtering or row removal.
    """

    def __init__(self,
                 input_file: str = "cleaned_ebird_file.pq",
                 output_file: str = "new_file.pq",
                 batch_size: int = 50000,
                 i95_coords: Optional[List] = i95_coords,
                 use_compression: str = 'snappy'):

        self.input_file = Path(input_file)
        self.output_file = Path(output_file)
        self.batch_size = batch_size
        self.use_compression = use_compression

        # Setup parquet file metadata
        self._setup_parquet_metadata()

        # Pre-process I-95 coordinates into optimized spatial structures
        self._setup_highway_geometry(i95_coords)

        # Statistics tracking
        self.total_rows_processed = 0
        self.batch_count = 0

    def _setup_parquet_metadata(self):
        """Get parquet file metadata for optimization"""
        try:
            # Read parquet metadata
            parquet_file = pq.ParquetFile(self.input_file)
            self.parquet_metadata = parquet_file.metadata
            self.parquet_schema = parquet_file.schema
            self.total_rows = self.parquet_metadata.num_rows

            logger.info(f"Parquet file info:")
            logger.info(f"  Total rows: {self.total_rows:,}")
            logger.info(f"  Number of row groups: {self.parquet_metadata.num_row_groups}")
            logger.info(f"  Columns: {len(self.parquet_schema)}")

            # Get file size
            self.total_file_size = self.input_file.stat().st_size / (1024**3)  # GB
            logger.info(f"  File size: {self.total_file_size:.2f} GB")

        except Exception as e:
            logger.error(f"Error reading parquet metadata: {e}")
            raise

    def _setup_highway_geometry(self, i95_coords):
        """Convert I-95 coordinates to optimized spatial structures"""
        if not i95_coords:
            raise ValueError("I-95 coordinates must be provided")

        logger.info("Setting up highway geometry with spatial indexing...")

        # Create LineString geometry from coordinates
        # i95_coords are (lat,lon) but LineString expects (lon,lat), so swap them
        self.highway_line = LineString([(lon, lat) for lat, lon in i95_coords])

        # Create GeoDataFrame for the highway with spatial index
        highway_gdf = gpd.GeoDataFrame([1], geometry=[self.highway_line], crs='EPSG:4326')

        # Convert to projected CRS for accurate distance calculations (UTM Zone 18N)
        self.highway_gdf_projected = highway_gdf.to_crs('EPSG:32618')
        self.highway_line_projected = self.highway_gdf_projected.geometry.iloc[0]

        logger.info("Highway geometry setup complete")

    def calculate_distances_vectorized(self, obs_gdf: gpd.GeoDataFrame) -> np.ndarray:
        """
        Vectorized distance calculation using GeoPandas
        This is the key optimization - processes all points at once
        """
        # Project observations to same CRS as highway (UTM Zone 18N)
        obs_projected = obs_gdf.to_crs('EPSG:32618')

        # Vectorized distance calculation to highway line
        distances_meters = obs_projected.geometry.distance(self.highway_line_projected)

        # Convert meters to miles
        distances_miles = distances_meters * 0.000621371

        return distances_miles.values

    def process_batch(self, batch_df: pd.DataFrame) -> pd.DataFrame:
        """Process batch to add I-95 distance column"""
        
        # Start with copy of original batch
        result_df = batch_df.copy()
        
        # Initialize distance column with NaN
        result_df['i95_distance'] = np.nan
        
        # Find rows with valid coordinates
        valid_mask = (
            batch_df['LATITUDE'].notna() & 
            batch_df['LONGITUDE'].notna()
        )
        
        if not valid_mask.any():
            logger.warning("No valid coordinates in batch")
            return result_df
        
        # Get valid coordinates
        valid_coords = batch_df[valid_mask].copy()
        
        # Convert to numeric if needed
        coord_cols = ['LATITUDE', 'LONGITUDE']
        for col in coord_cols:
            if valid_coords[col].dtype == 'object':
                valid_coords[col] = pd.to_numeric(valid_coords[col], errors='coerce')
        
        # Update mask after numeric conversion
        numeric_valid_mask = (
            valid_coords['LATITUDE'].notna() & 
            valid_coords['LONGITUDE'].notna()
        )
        
        if not numeric_valid_mask.any():
            logger.warning("No valid coordinates after numeric conversion")
            return result_df
        
        # Get final valid coordinates
        final_valid_coords = valid_coords[numeric_valid_mask]
        
        # Create GeoDataFrame from valid observations
        geometry = gpd.points_from_xy(final_valid_coords['LONGITUDE'], final_valid_coords['LATITUDE'])
        obs_gdf = gpd.GeoDataFrame(final_valid_coords, geometry=geometry, crs='EPSG:4326')
        
        # Calculate distances using vectorized operation
        distances = self.calculate_distances_vectorized(obs_gdf)
        
        # Add distances back to result dataframe at correct positions
        result_df.loc[final_valid_coords.index, 'i95_distance'] = distances
        
        return result_df

    def _process_one_batch(self, batch_df: pd.DataFrame) -> pd.DataFrame:
        """Process a single batch with logging and memory management"""
        self.batch_count += 1
        self.total_rows_processed += len(batch_df)

        if self.batch_count % 10 == 0:  # Log every 10th batch to reduce noise
            progress = (self.total_rows_processed / self.total_rows) * 100
            logger.info(f"Processing batch {self.batch_count} ({progress:.1f}% complete) "
                       f"with {len(batch_df)} rows")

        processed_batch = self.process_batch(batch_df)

        # Force garbage collection periodically
        if self.batch_count % 50 == 0:
            gc.collect()

        return processed_batch

    def _read_parquet_in_batches(self):
        """Generator to read parquet file in batches"""
        try:
            # Use pyarrow 
            parquet_file = pq.ParquetFile(self.input_file)

            # Read in batches using row groups
            for batch in parquet_file.iter_batches(batch_size=self.batch_size):
                # Convert to pandas DataFrame
                df = batch.to_pandas()
                yield df

        except Exception as e:
            logger.error(f"Error reading parquet in batches: {e}")
            # Fallback to pandas chunking
            logger.info("Falling back to pandas chunking...")
            try:
                # Read the entire file and chunk it manually per pandas doesn't support chunksize for parquet
                df = pd.read_parquet(self.input_file)
                for i in range(0, len(df), self.batch_size):
                    yield df.iloc[i:i+self.batch_size]
                del df  # Free memory
            except Exception as e2:
                logger.error(f"Fallback also failed: {e2}")
                raise

    def _write_parquet_batch(self, df: pd.DataFrame, is_first_batch: bool = False):
        """Write batch to parquet file efficiently"""
        if df.empty:
            return

        # Convert to Arrow Table for efficient writing
        table = pa.Table.from_pandas(df, preserve_index=False)

        if is_first_batch:
            # Create new file
            writer = pq.ParquetWriter(
                self.output_file,
                table.schema,
                compression=self.use_compression,
                use_dictionary=True  # Enable dictionary encoding
            )
            self._parquet_writer = writer

        # Write the batch
        self._parquet_writer.write_table(table)

    def run_pipeline(self) -> Dict[str, Any]:
        """Runs the complete pipeline to add I-95 distance column"""
        logger.info(f"Starting I-95 distance calculation pipeline: {self.input_file}")
        logger.info(f"Batch size: {self.batch_size:,}")
        logger.info(f"Expected batches: {(self.total_rows // self.batch_size) + 1}")

        start_time = pd.Timestamp.now()
        first_batch = True
        total_rows_saved = 0

        try:
            # Process file in batches
            for batch_df in self._read_parquet_in_batches():
                processed_batch = self._process_one_batch(batch_df)
                total_rows_saved += len(processed_batch)

                self._write_parquet_batch(processed_batch, is_first_batch=first_batch)
                first_batch = False

            # Close the parquet writer
            if hasattr(self, '_parquet_writer'):
                self._parquet_writer.close()
                logger.info(f"Saved {total_rows_saved:,} rows to {self.output_file}")
            else:
                logger.warning("No data to save")

        except Exception as e:
            logger.error(f"Pipeline error: {str(e)}")
            # Clean up partial file
            if self.output_file.exists():
                self.output_file.unlink()
            raise

        finally:
            # Cleanup
            if hasattr(self, '_parquet_writer'):
                try:
                    self._parquet_writer.close()
                except:
                    pass

        end_time = pd.Timestamp.now()
        processing_time = (end_time - start_time).total_seconds()

        # Calculate statistics
        stats = {
            'total_rows_processed': self.total_rows_processed,
            'total_rows_saved': total_rows_saved,
            'total_batches': self.batch_count,
            'input_file': str(self.input_file),
            'output_file': str(self.output_file),
            'input_file_size_gb': self.total_file_size,
            'output_file_size_gb': self.output_file.stat().st_size / (1024**3) if self.output_file.exists() else 0,
            'processing_time_seconds': processing_time,
            'processing_time_formatted': str(pd.Timedelta(seconds=processing_time)),
            'rows_per_second': self.total_rows_processed / processing_time if processing_time > 0 else 0,
            'compression_ratio': (self.total_file_size / (self.output_file.stat().st_size / (1024**3)))
                               if self.output_file.exists() and self.output_file.stat().st_size > 0 else 0
        }

        logger.info("I-95 distance calculation pipeline completed!")
        logger.info(f"Processing time: {stats['processing_time_formatted']}")
        logger.info(f"Rows per second: {stats['rows_per_second']:,.0f}")
        logger.info(f"Compression ratio: {stats['compression_ratio']:.2f}x")

        return stats

In [20]:
# Initialize calculator
calculator = I95DistanceCalculator(
    input_file="/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/stations.parquet",
    output_file="/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_with_distances.parquet",
    i95_coords=i95_coords,
    batch_size=50000,
    use_compression='snappy'
)

# Run the pipeline
stats = calculator.run_pipeline()

2025-07-18 13:44:57,959 - INFO - Parquet file info:
2025-07-18 13:44:57,959 - INFO -   Total rows: 952,074
2025-07-18 13:44:57,960 - INFO -   Number of row groups: 1
2025-07-18 13:44:57,960 - INFO -   Columns: 52
2025-07-18 13:44:57,960 - INFO -   File size: 0.07 GB
2025-07-18 13:44:57,960 - INFO - Setting up highway geometry with spatial indexing...
2025-07-18 13:44:58,032 - INFO - Highway geometry setup complete
2025-07-18 13:44:58,032 - INFO - Starting I-95 distance calculation pipeline: /Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/stations.parquet
2025-07-18 13:44:58,032 - INFO - Batch size: 50,000
2025-07-18 13:44:58,032 - INFO - Expected batches: 20
2025-07-18 13:46:11,341 - INFO - Processing batch 10 (52.5% complete) with 50000 rows
2025-07-18 13:47:32,247 - INFO - Processing batch 20 (100.0% complete) with 2074 rows
2025-07-18 13:47:32,654 - INFO - Saved 952,074 rows to /Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_with_distances.parquet
2025-07-18 

In [21]:
birds = pd.read_parquet("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_with_distances.parquet")
birds.shape, birds.columns

((952074, 16),
 Index(['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
        'OBSERVATION COUNT', 'STATE CODE', 'COUNTY', 'LOCALITY',
        'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
        'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'station_distances',
        'assigned_station', 'i95_distance'],
       dtype='object'))

In [22]:
na_count = birds.isna().sum()
print(f"Number of rows with NA values:\n{na_count}")

Number of rows with NA values:
TAXON CONCEPT ID                 0
COMMON NAME                      0
SCIENTIFIC NAME                  0
OBSERVATION COUNT                0
STATE CODE                       0
COUNTY                           0
LOCALITY                         0
LOCALITY TYPE                    0
LATITUDE                         0
LONGITUDE                        0
OBSERVATION DATE                 0
TIME OBSERVATIONS STARTED     3321
DURATION MINUTES             62586
station_distances                0
assigned_station                 0
i95_distance                     0
dtype: int64


### Add temporal details

In [23]:
# Convert 'OBSERVATION DATE' to datetime
birds['OBSERVATION DATE'] = pd.to_datetime(birds['OBSERVATION DATE'])
birds['TIME OBSERVATIONS STARTED'] = pd.to_datetime(birds['TIME OBSERVATIONS STARTED'], format='%H:%M:%S', errors='coerce') 

In [24]:
# Extract/Add date components
birds["year_record"] = birds["OBSERVATION DATE"].dt.year
birds["month_record"] = birds["OBSERVATION DATE"].dt.month
birds["day_record"] = birds["OBSERVATION DATE"].dt.day
birds["day_of_week"] = birds["OBSERVATION DATE"].dt.dayofweek  # 0=Monday, 6=Sunday
birds["is_weekend"] = (birds["OBSERVATION DATE"].dt.dayofweek >= 4).astype(int)  # 1 for Friday/Saturday/Sunday, 0 for weekdays

In [25]:
# Observation Hour
birds['hour_started'] = birds['TIME OBSERVATIONS STARTED'].apply(lambda x: x.hour)

In [26]:
# Migration flag
migration_months = [5, 6, 9, 10] 
birds["is_migration"] = birds["month_record"].isin(migration_months).astype(int)
print(birds.shape)

(952074, 23)


In [27]:
birds.shape, birds.columns

((952074, 23),
 Index(['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
        'OBSERVATION COUNT', 'STATE CODE', 'COUNTY', 'LOCALITY',
        'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
        'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'station_distances',
        'assigned_station', 'i95_distance', 'year_record', 'month_record',
        'day_record', 'day_of_week', 'is_weekend', 'hour_started',
        'is_migration'],
       dtype='object'))

In [28]:
birds.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_with_flags.parquet', compression='snappy')  

### Add taxonomy info
- "NorthAmericanBirds.csv": A file listing North American Bird species - common name, scientific name,  Family and Order - created from wikipedia page

In [29]:
taxo = pd.read_csv("/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/NorthAmericanBirds.csv")
print(taxo.shape)
print(taxo.columns)

(1283, 6)
Index(['Text', 'Common_Name', 'Scientific_Name', 'Order', 'Family', 'Class'], dtype='object')


In [30]:
taxo = taxo[['Scientific_Name', 'Order']]

In [31]:
taxo_merged = pd.merge(
    birds,
    taxo,
    left_on="SCIENTIFIC NAME",
    right_on="Scientific_Name",
    how="left"
)
print(taxo_merged.shape)
print(taxo_merged.columns)

(952074, 25)
Index(['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
       'OBSERVATION COUNT', 'STATE CODE', 'COUNTY', 'LOCALITY',
       'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
       'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'station_distances',
       'assigned_station', 'i95_distance', 'year_record', 'month_record',
       'day_record', 'day_of_week', 'is_weekend', 'hour_started',
       'is_migration', 'Scientific_Name', 'Order'],
      dtype='object')


In [32]:
na_count = taxo_merged.isna().sum()
print(f"Number of rows with NA values:\n{na_count}")

Number of rows with NA values:
TAXON CONCEPT ID                 0
COMMON NAME                      0
SCIENTIFIC NAME                  0
OBSERVATION COUNT                0
STATE CODE                       0
COUNTY                           0
LOCALITY                         0
LOCALITY TYPE                    0
LATITUDE                         0
LONGITUDE                        0
OBSERVATION DATE                 0
TIME OBSERVATIONS STARTED     3321
DURATION MINUTES             62586
station_distances                0
assigned_station                 0
i95_distance                     0
year_record                      0
month_record                     0
day_record                       0
day_of_week                      0
is_weekend                       0
hour_started                  3321
is_migration                     0
Scientific_Name              53799
Order                        53799
dtype: int64


### Add weather flag

In [33]:
taxo_merged["STATE CODE"].unique()

array(['US-VA', 'US-ME'], dtype=object)

In [34]:
# Pre-defined event windows for each state
EVENT_WINDOWS = {
    'US-ME': [
        ('2020-01-11', '2020-01-13'),
        ('2020-12-05', '2020-12-06'),
        ('2020-12-15', '2020-12-17'),
        ('2021-06-07', '2021-06-07'),
        ('2023-05-30', '2023-05-30'),
        ('2023-06-25', '2023-06-26'),
        ('2023-07-18', '2023-07-18'),
    ],
    'US-VA': [
        ('2020-05-28', '2020-05-28'),
        ('2020-08-04', '2020-08-04'),
        ('2020-12-14', '2020-12-17'),
        ('2021-07-09', '2021-07-09'),
        ('2021-08-18', '2021-08-18'),
        ('2021-09-21', '2021-09-21'),
        ('2021-07-21', '2021-07-21'),
        ('2022-01-27', '2022-01-29'),
        ('2023-03-02', '2023-03-03'),
        ('2023-03-31', '2023-04-01'),
        ('2023-06-02', '2023-06-07'),
        ('2023-09-23', '2023-09-24'),
    ]
}

In [35]:
def check_extreme_event(state: str, date_str: str) -> int:
    """
    Check if the given date falls within an extreme weather or poor air quality event for the specified state.

    Parameters:
    - state: "Maine" or "Virginia"
    - date_str: string in 'YYYY-MM-DD' format

    Returns:
    - 1 if date is within an event timeframe
    - 0 otherwise
    """
    try:
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
        if state not in EVENT_WINDOWS:
            return 0
        for start_str, end_str in EVENT_WINDOWS[state]:
            start = datetime.strptime(start_str, "%Y-%m-%d").date()
            end = datetime.strptime(end_str, "%Y-%m-%d").date()
            if start <= date <= end:
                return 1
        return 0
    except ValueError:
        raise ValueError("Invalid date format. Use YYYY-MM-DD.")

In [36]:
# add weather flag and verify the function works
taxo_merged['extreme_weather'] = taxo_merged.apply(lambda row: check_extreme_event(row['STATE CODE'], row['OBSERVATION DATE'].strftime('%Y-%m-%d')), axis=1)
print(taxo_merged.columns)
print(taxo_merged['extreme_weather'].value_counts())

Index(['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
       'OBSERVATION COUNT', 'STATE CODE', 'COUNTY', 'LOCALITY',
       'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
       'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'station_distances',
       'assigned_station', 'i95_distance', 'year_record', 'month_record',
       'day_record', 'day_of_week', 'is_weekend', 'hour_started',
       'is_migration', 'Scientific_Name', 'Order', 'extreme_weather'],
      dtype='object')
extreme_weather
0    945038
1      7036
Name: count, dtype: int64


### Key feature clean up - NA rows
Previouse EDA showed 41K non-numeric target rows present.<br>
TIME OBSERVATION STARTED is a key feature.  Will drop rows if NA

In [37]:
print("Before cleaning:", taxo_merged.shape)
# Cleaning up 41K nonnumeric data ("X") from our target
taxo_merged['OBSERVATION COUNT'] = pd.to_numeric(taxo_merged['OBSERVATION COUNT'], errors='coerce')
# Dropping non-numeric 'OBSERVATION COUNT' rows
taxo_merged = taxo_merged.dropna(subset=['OBSERVATION COUNT', 'TIME OBSERVATIONS STARTED'])
print("After cleaning:", taxo_merged.shape)

Before cleaning: (952074, 26)
After cleaning: (928224, 26)


In [38]:
na_count = taxo_merged.isna().sum()
print(f"Number of rows with NA values:\n{na_count}")

Number of rows with NA values:
TAXON CONCEPT ID                 0
COMMON NAME                      0
SCIENTIFIC NAME                  0
OBSERVATION COUNT                0
STATE CODE                       0
COUNTY                           0
LOCALITY                         0
LOCALITY TYPE                    0
LATITUDE                         0
LONGITUDE                        0
OBSERVATION DATE                 0
TIME OBSERVATIONS STARTED        0
DURATION MINUTES             58124
station_distances                0
assigned_station                 0
i95_distance                     0
year_record                      0
month_record                     0
day_record                       0
day_of_week                      0
is_weekend                       0
hour_started                     0
is_migration                     0
Scientific_Name              52869
Order                        52869
extreme_weather                  0
dtype: int64


### Add traffic and noise
Requires "i95_traffic_va_me_small_8am.csv": A file listing each noise observation stations with the detected daily noise at 8 am in years 2020 through 2023


In [39]:
traffic = pd.read_csv('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/i95_traffic_va_me_small_8am.csv', 
                      dtype={'station_id': str})
print(traffic.shape)
print(traffic.columns)

(387000, 45)
Index(['record_type', 'state_code', 'f_system', 'station_id', 'travel_dir',
       'travel_lane', 'year_record', 'month_record', 'day_record',
       'day_of_week', 'hour_00', 'hour_01', 'hour_02', 'hour_03', 'hour_04',
       'hour_05', 'hour_06', 'hour_07', 'hour_08', 'hour_09', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23', 'restrictions', 'latitude', 'longitude', 'station_location',
       'state', 'daily_avg_noise', 'peak_hour_noise', 'overnight_noise',
       '8am_noise', 'rush_hour_noise', 'total_daily_volume'],
      dtype='object')


In [40]:
traffic = traffic[['state_code', 'station_id', 'year_record', 'month_record', 'day_record',
       'daily_avg_noise', 'peak_hour_noise', 'overnight_noise', 
       'rush_hour_noise', 'total_daily_volume', '8am_noise']]

In [41]:
traffic.year_record.value_counts()

year_record
2022    98813
2023    97496
21      96216
2020    94475
Name: count, dtype: int64

In [42]:
print(traffic.shape)
# Find year_record = 21
condition = traffic['year_record'] == 21

# Select the rows and the specific column, then assign the new value
traffic.loc[condition, 'year_record'] = 2021

print("\nDataFrame after replacement (Method 1 - Boolean Indexing):")
print(traffic.shape)

(387000, 11)

DataFrame after replacement (Method 1 - Boolean Indexing):
(387000, 11)


In [43]:
traffic.year_record.value_counts()

year_record
2022    98813
2023    97496
2021    96216
2020    94475
Name: count, dtype: int64

In [44]:
na_count = traffic.isna().sum()
print(f"Number of rows with NA values:\n{na_count}")

Number of rows with NA values:
state_code               0
station_id               0
year_record              0
month_record             0
day_record               0
daily_avg_noise        141
peak_hour_noise        141
overnight_noise        295
rush_hour_noise        168
total_daily_volume       0
8am_noise             1593
dtype: int64


In [45]:
taxo_merged.shape, taxo_merged.columns

((928224, 26),
 Index(['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
        'OBSERVATION COUNT', 'STATE CODE', 'COUNTY', 'LOCALITY',
        'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
        'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'station_distances',
        'assigned_station', 'i95_distance', 'year_record', 'month_record',
        'day_record', 'day_of_week', 'is_weekend', 'hour_started',
        'is_migration', 'Scientific_Name', 'Order', 'extreme_weather'],
       dtype='object'))

Resolving traffic['station'] and taxo_merged['assigned_station'] values mismatch by adding leading 0s to traffic['station']

In [46]:
traffic['station'] = traffic[['station_id']].astype(str)
traffic['station'] = traffic['station'].str.zfill(6)
traffic['station'].value_counts().sort_index()


station
030209    4334
030210    4335
040007    1431
040046    1459
040052    1392
          ... 
840062    1437
840066    1429
940050    1438
940447    1439
940762    1427
Name: count, Length: 254, dtype: int64

In [47]:
taxo_merged[['assigned_station']]=taxo_merged[['assigned_station']].astype(str)

In [48]:
common = set(traffic['station']).intersection(taxo_merged['assigned_station'])
print(f"Common stations: {len(common)}")  

Common stations: 33


In [49]:
# Check duplicates in traffic
traffic_dupes = traffic.groupby(['station', 'year_record', 'month_record', 'day_record']).size()
print("Traffic duplicates:", traffic_dupes[traffic_dupes > 1].head())

Traffic duplicates: station  year_record  month_record  day_record
030209   2020         1             1             3
                                    2             3
                                    3             3
                                    4             3
                                    5             3
dtype: int64


Traffic data hs duplicate rows.  Removing duplicates before merging

In [50]:
traffic_deduped = traffic.drop_duplicates(
    subset=['station', 'year_record', 'month_record', 'day_record'])
traffic_deduped.shape

(335335, 12)

In [51]:
print("before merge:", taxo_merged.shape)
merged_df = pd.merge(
    taxo_merged,
    traffic_deduped,
    left_on=['assigned_station', 'year_record', 'month_record', 'day_record'],
    right_on=['station', 'year_record', 'month_record', 'day_record'],
    how="left"
)
print("after merge:", merged_df.shape)
merged_df.columns

before merge: (928224, 26)
after merge: (928224, 35)


Index(['TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME',
       'OBSERVATION COUNT', 'STATE CODE', 'COUNTY', 'LOCALITY',
       'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
       'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'station_distances',
       'assigned_station', 'i95_distance', 'year_record', 'month_record',
       'day_record', 'day_of_week', 'is_weekend', 'hour_started',
       'is_migration', 'Scientific_Name', 'Order', 'extreme_weather',
       'state_code', 'station_id', 'daily_avg_noise', 'peak_hour_noise',
       'overnight_noise', 'rush_hour_noise', 'total_daily_volume', '8am_noise',
       'station'],
      dtype='object')

Clean up duplicative colums created during the merges.

In [52]:
merged_df = merged_df[['OBSERVATION COUNT','COMMON NAME', 'SCIENTIFIC NAME', 'Order',
        'STATE CODE', 'COUNTY', 'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 
        'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'i95_distance', 
        'station_distances', 'assigned_station', 'year_record', 'month_record', 
        'day_record', 'hour_started', 'day_of_week', 'is_weekend', 'is_migration', 
        'extreme_weather', 'daily_avg_noise', 'peak_hour_noise', 'overnight_noise', 
        'rush_hour_noise', 'total_daily_volume', '8am_noise',
       ]]
merged_df.shape

(928224, 28)

In [53]:
na_count = merged_df.isna().sum()
print(f"Number of rows with NA values:\n{na_count}")

Number of rows with NA values:
OBSERVATION COUNT                 0
COMMON NAME                       0
SCIENTIFIC NAME                   0
Order                         52869
STATE CODE                        0
COUNTY                            0
LOCALITY TYPE                     0
LATITUDE                          0
LONGITUDE                         0
TIME OBSERVATIONS STARTED         0
DURATION MINUTES              58124
i95_distance                      0
station_distances                 0
assigned_station                  0
year_record                       0
month_record                      0
day_record                        0
hour_started                      0
day_of_week                       0
is_weekend                        0
is_migration                      0
extreme_weather                   0
daily_avg_noise              212923
peak_hour_noise              212923
overnight_noise              212959
rush_hour_noise              212942
total_daily_volume           2129

8am_noise is a key feature.  Dropping rows if 8am_noise is NA.

In [54]:
cleaned_df = merged_df.dropna(subset=['8am_noise'])
print('Final DF Shape:', cleaned_df.shape)
na_count = cleaned_df.isna().sum()
print(f"Number of rows with NA values:\n{na_count}")

Final DF Shape: (715180, 28)
Number of rows with NA values:
OBSERVATION COUNT                0
COMMON NAME                      0
SCIENTIFIC NAME                  0
Order                        40480
STATE CODE                       0
COUNTY                           0
LOCALITY TYPE                    0
LATITUDE                         0
LONGITUDE                        0
TIME OBSERVATIONS STARTED        0
DURATION MINUTES             46723
i95_distance                     0
station_distances                0
assigned_station                 0
year_record                      0
month_record                     0
day_record                       0
hour_started                     0
day_of_week                      0
is_weekend                       0
is_migration                     0
extreme_weather                  0
daily_avg_noise                  0
peak_hour_noise                  0
overnight_noise                 28
rush_hour_noise                  0
total_daily_volume            

In [55]:
cleaned_df.shape

(715180, 28)

In [56]:
cleaned_df.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_test_ready_no_filter.parquet', compression='snappy')

### Creating test subsets

In [57]:
filtered_8am = cleaned_df[cleaned_df['hour_started'] == 8]
print("Filtered DataFrame shape (8 AM observations):", filtered_8am.shape)
filtered_8am.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_8am.parquet', compression='snappy')

filtered_8am_weekend = filtered_8am[filtered_8am['is_weekend'] == 1]
print("Filtered DataFrame shape (8 AM weekend observations):", filtered_8am_weekend.shape)  
filtered_8am_weekend.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_8am_weekend.parquet', compression='snappy')

filtered_weekend = cleaned_df[cleaned_df['is_weekend'] == 1]
print("Filtered DataFrame shape (weekend observations):", filtered_weekend.shape)
filtered_weekend.to_parquet('/Users/sooneui/Documents/UCB_MIDS/Data/Capstone_Data/ETL/birds_weekend.parquet', compression='snappy')

Filtered DataFrame shape (8 AM observations): (100387, 28)
Filtered DataFrame shape (8 AM weekend observations): (52106, 28)
Filtered DataFrame shape (weekend observations): (349112, 28)


In [58]:
cleaned_df.columns

Index(['OBSERVATION COUNT', 'COMMON NAME', 'SCIENTIFIC NAME', 'Order',
       'STATE CODE', 'COUNTY', 'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE',
       'TIME OBSERVATIONS STARTED', 'DURATION MINUTES', 'i95_distance',
       'station_distances', 'assigned_station', 'year_record', 'month_record',
       'day_record', 'hour_started', 'day_of_week', 'is_weekend',
       'is_migration', 'extreme_weather', 'daily_avg_noise', 'peak_hour_noise',
       'overnight_noise', 'rush_hour_noise', 'total_daily_volume',
       '8am_noise'],
      dtype='object')

In [59]:
aggregated_df = cleaned_df.groupby(['assigned_station', 'year_record', 'month_record', 'day_record'])['OBSERVATION COUNT'].sum().reset_index()
aggregated_df_weekend = filtered_weekend.groupby(['assigned_station', 'year_record', 'month_record', 'day_record'])['OBSERVATION COUNT'].sum().reset_index()
aggregated_df_8am = filtered_8am.groupby(['assigned_station', 'year_record', 'month_record', 'day_record'])['OBSERVATION COUNT'].sum().reset_index()

In [60]:
print("aggregated_df (unfiltered):", aggregated_df.shape)
print("aggregated_df_weekend:", aggregated_df_weekend.shape)
print("aggregated_df_8am:", aggregated_df_8am.shape)


aggregated_df (unfiltered): (22735, 5)
aggregated_df_weekend: (10251, 5)
aggregated_df_8am: (8958, 5)


In [61]:
cleaned_df.year_record.unique()

array([2020, 2021, 2022, 2023], dtype=int32)